In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
plt.style.use('ggplot')
import researchpy as rp
from scipy import stats
%matplotlib inline
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant


In [25]:
df = pd.read_csv('data/clean_df')

In [9]:
X = np.array(df[['A1Cresult', 'num_medications', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'number_inpatient']].values)



X


array([['None', 1, 1, 41, 0, 0],
       ['None', 18, 3, 59, 0, 0],
       ['None', 13, 2, 11, 5, 1],
       ...,
       ['None', 9, 1, 53, 0, 0],
       ['None', 21, 10, 45, 2, 1],
       ['None', 3, 6, 13, 3, 0]], dtype=object)

In [18]:
X = np.array(df[['num_medications', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'number_inpatient']].values)
y = np.array(df['readmitted_<30d'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=0)
logit = sm.Logit(y_train, X_train)
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.385502
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                76521
Model:                          Logit   Df Residuals:                    76516
Method:                           MLE   Df Model:                            4
Date:                Fri, 28 Jun 2019   Pseudo R-squ.:                -0.08855
Time:                        14:09:51   Log-Likelihood:                -29499.
converged:                       True   LL-Null:                       -27099.
                                        LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0557      0.002    -33.959      0.000      -0.059      -0.053
x2            -0.0135      0.

In [ ]:

# convert data type of nominal features in dataframe to 'object' type
i = ['encounter_id', 'patient_nbr', 'gender', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',\
          'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', \
          'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose','miglitol', \
          'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', \
          'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', \
          'age', 'A1Cresult', 'max_glu_serum', 'level1_diag1', 'level1_diag2', 'level1_diag3', 'level2_diag1', 'level2_diag2', 'level2_diag3']

df[i] = df[i].astype('object')

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [19]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [20]:
clf.feature_importances_

array([0.27266418, 0.11977215, 0.46558271, 0.08360281, 0.05837816])

In [22]:
clf.predict(X_test)

array([False, False, False, ..., False, False, False])

In [23]:
clf.score

<bound method ClassifierMixin.score of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)>